In [15]:
import requests  
from lxml import etree 
import pickle
import os
from IPython.core.display import display, HTML
import timeit
import jieba

class MySearcherC8V0:
    """
    第七次课升级的搜索类版本：
    1、__init__()初始化过程加载自定义分词词典
    2、build_cache()改用cut_for_search进行分词
    3、search()对查询分词
    4、search()对分词结果取posting
    5、search()对posting lists进行合并(交集)
    6、build_cache()将posting保存格式改成只用doc_id(方便集合运算)
    7、rank()实现对候选文档打分排序
    8、score()实现对查询中包含的多词统计词频计分
    """
    def __init__(self, scale=1):
        self.docs = []
        self.load_data()
        self.docs *= scale
        self.cache = {}
        self.vocab = set()
        self.lower_preprocess()
        self.build_cache()
        jieba.load_userdict('dict.txt')
    
    def build_cache(self):
        doc_id = 0
        for doc in self.docs:
            doc_word_set = set()
            for word in jieba.cut_for_search(
                doc[3]
            ):
                if word not in doc_word_set:
                    result_item = doc_id
                    if word not in self.cache:
                        self.cache[word] = set([result_item])
                    else:
                        self.cache[word].add(result_item)
                    self.vocab.add(word)
                    doc_word_set.add(word)
            doc_id += 1
    
    def search(self, query):
        result = None
        for keyword in jieba.cut(query.lower()):
            if keyword in self.cache:
                if result is None:
                    result = self.cache[keyword]
                else:
                    result = result & self.cache[keyword]
            else:
                result = set([])
                break
                
        if result is None:
            result = set([])
        
        sorted_result = self.rank(query, result)
        return sorted_result
                    
    def lower_preprocess(self):
        for doc_id in range(len(self.docs)):
            self.docs[doc_id].append(
                (self.docs[doc_id][1] 
                 + ' ' 
                 + self.docs[doc_id][2]).lower()
            )
    
    def simple_test(self):
        assert(len(self.search('tiktok')) > 1)
    
    def load_data(self):
        data_filename = 'news_list.dat'
        if os.path.exists(data_filename):
            with open(data_filename,'rb') as f:
                self.docs += pickle.load(f)
#                 self.docs = self.docs + pickle.load(f)
        else:
            url = 'http://news.163.com/special/0001386F/rank_tech.html'  
            headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36 Edg/85.0.564.63'}
            r = requests.get(url, headers=headers)  
            sel = etree.HTML(r.text) 
            link_set = set()
            news_list = []
            count = 0
            for item in sel.xpath('//td/a'):  
                title = item.text
                link = item.attrib['href']
            #     print(link, title)
                if link not in link_set:
                    r = requests.get(link, headers=headers)  
                    sel = etree.HTML(r.text)  
                    text_block = sel.xpath('//div[@id="endText"]') 
                #     print(''.join(text_block[0].itertext()))
                    if text_block:
                        content = ''.join(text_block[0].xpath('./p/text()'))
                        title = sel.xpath('//h1/text()')[0]
                        self.docs.append([link, title, content])
                    link_set.add(link)
                count += 1
                if count % 15 == 0:
                    print(count, 'processed.')
            with open(data_filename,'wb') as f:
                pickle.dump(self.docs, f)
    
    def highlight(self, text, keyword):
        idx = text.lower().find(keyword.lower())
        result = text
        if idx >= 0:
            ori_word = text[idx:idx+(len(keyword))]
            result = text.replace(ori_word, '<span style="color:red";>{}</span>'.format(ori_word))
        return result
    
    def rank(self, query, result_set):
        result = []
        for doc_id in result_set:
            result.append([doc_id, 
                self.score(self.docs[doc_id],
                          query)])
        result.sort(key=lambda x: x[1], reverse=True)
        return result       
    
    def score(self, item, query):
        score = 0
        #todo cut
        for keyword in jieba.cut(query):
            score += item[1].lower().count(keyword.lower()) * 5 \
                      + item[2].lower().count(keyword.lower()) * 3
        return score
    
    def render_search_result(self, keyword):
        count = 0
        for item in self.search(keyword):
            count += 1
            display(HTML('{} [{}] {}'.format(count, item[1], 
                self.highlight(self.docs[item[0]][1], keyword) + '<br/>' + self.docs[item[0]][2])))

In [16]:
%%time
searcher_v0 = MySearcherC8V0()

Wall time: 6.48 s


In [ ]:
searcher_v0.render_search_result('华为手机')

In [6]:
class MySearcherC8V1(MySearcherC8V0):
    """
    多关键词摘要和高亮
    """
    def highlight(self, text, keyword):
        idx = text.lower().find(keyword.lower())
        result = text
        if idx >= 0:
            ori_word = text[idx:idx+(len(keyword))]
            result = text.replace(ori_word, '<span style="color:red";>{}</span>'.format(ori_word))
        return result

In [7]:
searcher_v1 = MySearcherC8V1()

In [14]:
searcher_v1.render_search_result('华为手机')

In [27]:
def highlight(item, query, sidelen=12):
    result = ''
    positions = []
    content_lower = item[2].lower()
    for keyword in jieba.cut(query):
        idx = content_lower.find(keyword.lower())
        positions.append(idx)
    print(positions)
    positions.sort()
    print(positions)
    segments = []
    i = 0
    while i < len(positions):
        start_pos = max(positions[i] - sidelen, 0)
        end_pos = min(positions[i] + sidelen, len(content_lower))
        while (i < len(positions) - 1 
            and positions[i+1] - positions[i] <= 2 * sidelen):
            end_pos = min(positions[i+1] + sidelen, len(content_lower))
            i += 1  
        start_ddd = '...' if start_pos > 0 else ''
        end_ddd = '...' if end_pos < len(content_lower) else ''
        segments.append(start_ddd 
                        + item[2][start_pos:end_pos]
                        + end_ddd)
        i += 1
    print(segments)
    result = item[1] + '<br/>' + ''.join(segments)
#     idx = item[1].lower().find(query.lower())
#     result = item[1]
#     if idx >= 0:
#         ori_word = text[idx:idx+(len(query))]
#         result = text.replace(ori_word, '<span style="color:red";>{}</span>'.format(ori_word))
    return result

print(highlight([
    '',
    '华为Mate40来了 硬刚iPhone12！买哪个？网友吵起来了', 
    '华为Mate40来了，硬刚iPhone12！该买哪一个？网友吵起来了） 每经编辑 何小桃这个10月，华为Mate 40系列手机将与iPhone 12系列手机正面对决'
], '对决华为系列手机'))

[80, 0, 59, 61]
[0, 59, 61, 80]
['华为Mate40来了，硬...', '...0月，华为Mate 40系列手机将与iPhone 12系列手机正面对决']
华为Mate40来了 硬刚iPhone12！买哪个？网友吵起来了<br/>华为Mate40来了，硬......0月，华为Mate 40系列手机将与iPhone 12系列手机正面对决
